In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [5]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("augment_train.csv")
df_test = pd.read_csv("evaluation.csv")

In [8]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install -q tensorflow_hub
    import tensorflow_hub as hub

try:
    import tensorflow_text as text
except ModuleNotFoundError:
    %pip install -q -U "tensorflow-text==2.8.*"
    %pip install -q tf-models-official==2.7.0
    import tensorflow_text as text

try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np
import math as m

     |████████████████████████████████| 4.9 MB 8.2 MB/s 
     |████████████████████████████████| 1.8 MB 8.2 MB/s 
     |████████████████████████████████| 99 kB 12.2 MB/s 
     |████████████████████████████████| 1.1 MB 65.1 MB/s 
     |████████████████████████████████| 352 kB 81.6 MB/s 
     |████████████████████████████████| 116 kB 76.4 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 1.3 MB 57.3 MB/s 
     |████████████████████████████████| 238 kB 76.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=d75d3a4065a02ff03a795da60b054cb03e0381778a703f333867a2b11d9a2eb6
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [10]:
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/2"

In [21]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hat_stack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hat_stack

In [12]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [15]:
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
bert_layer1 = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name="textEncoder")
bert_layer2 = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name="ReasonEncoder")
def encoder1(name):
    inputs = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
    
    encoder_inputs = preprocessing_layer(inputs)
    encoder_outputs = bert_layer1(encoder_inputs)
    net = encoder_outputs['pooled_output']
    a = tfl.Dropout(0.3)(net)
    a = tfl.Dense(128, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

def encoder2(name):
    inputs = tf.keras.layers.Input(shape=(), dtype=tf.string, name="reason")
    
    encoder_inputs = preprocessing_layer(inputs)
    encoder_outputs = bert_layer2(encoder_inputs)
    net = encoder_outputs['pooled_output']
    a = tfl.Dropout(0.3)(net)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [16]:
def get_model():
    #textEncoder
    inputText = tf.keras.layers.Input(shape=(), dtype=tf.string)
    text_embedd = encoder1("textBertEncoder")(inputText)

    #reasonEncoder
    inputReason = tf.keras.layers.Input(shape=(), dtype=tf.string)
    reason_embedd = encoder2("reasonBertEncoder")(inputReason)

    combined_features = tfl.concatenate([text_embedd, reason_embedd], name = 'merge_features')
    print(combined_features.shape)
    combined_features = tfl.Dropout(0.2)(combined_features)
    combined_features = tfl.Dense(16, activation = 'linear', kernel_regularizer=l2(1e-3))(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(4, activation = 'linear', kernel_regularizer=l2(1e-3))(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(1, activation = 'sigmoid')(combined_features)
    
    model = Model(inputs = [inputText, inputReason], outputs = [combined_features], name="BertModel")
    return model

In [17]:
model = get_model()

(None, 64)


In [18]:
model.summary()

Model: "BertModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 model (Functional)             (None, 32)           14373729    ['input_1[0][0]']                
                                                                                                  
 model_1 (Functional)           (None, 32)           14348513    ['input_2[0][0]']                
                                                                                          

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss = "binary_crossentropy", metrics = [tf.metrics.BinaryAccuracy()])

In [20]:
eval_df = df_test[df_test['label'] == 1]

In [ ]:
loss_history = model.fit(siam_gen([df["clean_text"], df["reason"]], 128),
                         validation_data = siam_gen([eval_df["clean_text"].values, eval_df["reason"].values], 128),
                         steps_per_epoch=50, validation_steps=10, epochs = 50, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/50


In [ ]:
from sklearn.metrics import brier_score_loss as brier_loss
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

In [ ]:
y_preds = model.predict([df['clean_text'], df['reason']])
accuracy(df['label'], y_preds)



1.0

In [ ]:
y_hat = (y_preds >= 0.5).astype(int)

In [ ]:
print("Brier Loss                       : ", brier_loss(df['label'], y_preds))
print("Precision Score                  : ", precision_score(df['label'], y_hat))
print("Recall Score                     : ", recall_score(df['label'], y_hat))
print("F1 Score                         : ", f1_score(df['label'], y_hat))

In [ ]:
y_preds = model.predict([df_test['clean_text'], df_test['reason']])
accuracy(df_test['label'], y_preds)

0.513

In [ ]:
y_hat = (y_preds >= 0.5).astype(int)

In [ ]:
print("Brier Loss                       : ", brier_loss(df_test['label'], y_preds))
print("Precision Score                  : ", precision_score(df_test['label'], y_hat))
print("Recall Score                     : ", recall_score(df_test['label'], y_hat))
print("F1 Score                         : ", f1_score(df_test['label'], y_hat))
print("Roc AUC Score                    : ", roc_auc_score(df_test['label'], y_preds))
print("BCE Loss                         : ", log_loss(df_test['label'], y_preds))

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(df_test['label'], y_preds)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc[2],
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()